In [ ]:
!pip install openai langchain google-search-results

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = '<your key here>'
os.environ['SERPAPI_API_KEY'] = '<your key here>'

In [9]:
# app.py
from langchain import LLMChain, OpenAI, SerpAPIWrapper
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent

from lcserve import serving


# @serving
def ask(input: str) -> str:
    search = SerpAPIWrapper()
    tools = [
        Tool(
            name="Search",
            func=search.run,
            description="useful for when you need to find recipes",
        )
    ]
    # prefix = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:"""
    suffix = """Begin!

    Question: {input}
    {agent_scratchpad}"""

    prompt = ZeroShotAgent.create_prompt(
        tools,
        suffix=suffix,
        input_variables=["input", "agent_scratchpad"],
    )

    print(prompt.template)

    llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
    tool_names = [tool.name for tool in tools]
    agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent, tools=tools, verbose=True
    )

    return agent_executor.run(input)

In [11]:
output = ask('1. find the url for a good sushi recipe\n 2. verify it doesn\'t cause 404 error\n 3. Return the URL')

Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to find recipes

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

    Question: {input}
    {agent_scratchpad}


> Entering new AgentExecutor chain...

Thought: I need to find a good sushi recipe and make sure it works
Action: Search
Action Input: sushi recipe
Observation: Check out our best sushi recipes for ideas—you'll only get better with practice, so get to rolling! It's ALL about the rice.
Thought: I need to make sure the URL works
Action: Search
Action Input: URL of sushi recipe
Observ